In [10]:
import pandas as pd
from ast import literal_eval

In [ ]:
!pip install --upgrade torch torchvision torchaudio transformers

In [12]:
#read airline reviews with keywords
df_full = pd.read_csv("BA_Airline_BERTopicReviews_WithKeywords.csv")

list_columns = ["zs_preds", "bt_preds", "hybrid_labels", "topic_keywords"]
for col in list_columns:
    df_full[col] = df_full[col].apply(literal_eval)


In [13]:
def build_prompt(row):
    categories = ", ".join(row["hybrid_labels"])
    keywords   = ", ".join(row["topic_keywords"])
    return f"""You are an airline quality expert. Given the following airline review analysis:

- Categories: [{categories}]
- Keywords: {keywords}

Suggest one concise, specific, non-repetitive improvement the airline can implement based on all of the above.
"""

df_full["recommendation_prompt"] = df_full.apply(build_prompt, axis=1)

In [15]:
from transformers import pipeline

generator = pipeline("text2text-generation", model="google/flan-t5-xl")


config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


In [16]:
df_full["recommendation"] = df_full["recommendation_prompt"].apply(
    lambda prompt: generator(prompt, max_new_tokens=100)[0]["generated_text"]
)

In [17]:
#drop predicted and keyword columns and save to CSV
columns_to_drop = ["processed", "zs_preds", "bt_preds", "topic_keywords", "related_terms", "recommendation_prompt"]
df_recommendations = df_full.drop(columns=columns_to_drop, errors="ignore")
df_recommendations.to_csv("BA_Airline_Reviews_Recommendations.csv", index=False)